## Imports

In [1]:
import tensorflow as tf
from keras import datasets, layers, models
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import os
from time import sleep
import cv2
import pandas as pd


2023-02-22 22:12:06.174802: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-22 22:12:09.569835: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-22 22:12:09.572181: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-22 22:12:09.572253: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

## Making the dataset

In [2]:
# Capture video from webcam and make dataset
class MakeDataset():
    def __init__(self):
        import cv2
        import os
        
        self.video = cv2.VideoCapture(0)
        self.count = 0
        self.person_name = input("Enter person name: ")
        # self.num_images = int(input("Enter number of images: "))
        self.path = "data/" + self.person_name
        if not os.path.exists(self.path):
            os.makedirs(self.path)
    def  make(self):
        while True:
            _, frame = self.video.read()
            cv2.imshow("frame", frame)
            if cv2.waitKey(1) & 0xFF == ord('s'):
                self.count += 1
                cv2.imwrite(self.path + "/" + self.person_name + '_' + str(self.count) + ".jpg", frame)
                print(f'Image saved: {self.count}', end='\r')
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            if self.count == 50:
                break
    def __final__(self):
        self.video.release()
        cv2.destroyAllWindows()

# MakeDataset().make()

## Loading the data

### 1. Listing images in the dataset directory

In [2]:
# # Devasheesh's faces
# deva_faces = './data/devasheesh/'
# deva_faces = os.listdir(deva_faces)
# deva_faces = [deva_faces[i] for i in range(len(deva_faces)) if deva_faces[i].endswith('.jpg')]

# # Swarnim's faces
# swar_faces = './data/swarnim/'
# swar_faces = os.listdir(swar_faces)
# swar_faces = [swar_faces[i] for i in range(len(swar_faces)) if swar_faces[i].endswith('.jpg')]

# # Negative faces
# neg_faces = './data/negative-faces/'
# neg_faces = os.listdir(neg_faces)
# neg_faces = [neg_faces[i] for i in range(len(neg_faces)) if neg_faces[i].endswith('.jpg')]

# Making a function to list all the images in a folder
def list_images(path):
    image_files = []
    for root, _, files in os.walk(path):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_files.append(os.path.join(root, file))
    image_files.sort()
    return image_files

deva_faces = list_images('./data/Devasheesh/')
swar_faces = list_images('./data/Swarnim/')
neg_faces = list_images('./data/negative-faces/')

### 2. Making Train and Test Objects

In [4]:
# function to extract the face from the image
# using the Haar Cascade Classifier
def extract_frontal_face_harr(image_ndarray, grayscale=True, size=(150, 150)):
    # Load the cascade
    face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
    # Convert into grayscale
    if grayscale:
        image_gray = cv2.cvtColor(image_ndarray, cv2.COLOR_BGR2GRAY)
    else:
        image_gray = image_ndarray
    # Detect faces
    faces_cord = face_cascade.detectMultiScale(image_gray, 1.3, 5)
    # Return the face or None if not found
    if len(faces_cord) == 0:
        return None, None
    # Extract the face
    (x, y, w, h) = faces_cord[0]
    # Resize the image to 150x150
    image_gray_resized = cv2.resize(image_gray[y:y+w, x:x+h], size)
    # Return only the face part of the image
    return image_gray_resized, faces_cord

# using DNN
modelFile = "dnn/res10_300x300_ssd_iter_140000.caffemodel"
configFile = "dnn/deploy.prototxt.txt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

def extract_frontal_face(image_ndarray, grayscale=False, size=(150, 150)):
    # resize the image to 300x300 for the DNN model
    h, w = 300, 300
    image_ndarray = cv2.resize(image_ndarray, (h, w))
    # Convert into blob
    blob = cv2.dnn.blobFromImage(image_ndarray, 1.0, (h, w), (104.0, 177.0, 123.0))
    # Convert into grayscale
    if grayscale:
        image_ndarray = cv2.cvtColor(image_ndarray, cv2.COLOR_BGR2GRAY)
    # Detect faces
    net.setInput(blob)
    detections = net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            # Get the coordinates of the bounding box
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x2, y2) = box.astype("int")

            # Extract the face ROI (region of interest) from the image
            face = image_ndarray[y:y2, x:x2]
            return cv2.resize(face, size), (x, y, x2, y2)
        else:
            return None, None


In [5]:
# function to make the image size uniform and make train and test object with labels
class dataframe():
    def __init__(self):
        self.deva_faces = deva_faces
        self.swar_faces = swar_faces
        self.neg_faces = neg_faces
        self.images, self.labels = list(), list()

        self.total_images = len(deva_faces)+len(swar_faces)+len(neg_faces)
        self.labels_list = ['Unknown', 'Devasheesh', 'Swarnim']

        print('Total images: ', self.total_images)

        self.datagen = ImageDataGenerator(
            rotation_range=40,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.2, # randomly shift images vertically (fraction of total height)
            shear_range=0.2,    # set range for random shear
            zoom_range=0.2,    # set range for random zoom
            horizontal_flip=True,   # randomly flip images
            fill_mode='nearest' # set mode for filling points outside the input boundaries
        )

    def append_dataframe(self, images_list, label, augment=False, grayscale=False):
        count = 1
        for image_adr in images_list:
            # read the image
            image = cv2.imread(image_adr)
            # extract the face
            face, _ = extract_frontal_face(image, grayscale=grayscale, size=(300, 300))

            print('Count: ', count, ' | Image address: ', image_adr, ' | Label: ',self.labels_list[label], ' | Image shape: ', image.shape)
            if face is not None:
                # print('Face type: ', type(face))
                # print('Image shape: ', image.shape)
                # print('Face shape: ', face.shape)
                # print('Face: ', plt.imshow(face))

                # append the face image to the list
                if augment:
                    # using the image data generator to augment the images
                    i = 0
                    for batch in self.datagen.flow(face.reshape(1, 300, 300, 3), batch_size=1):
                        # append the image to the list
                        self.images.append(batch[0])
                        self.labels.append(label)
                        i += 1
                        if i > 10:
                            break
                    count += 1

                else:
                    self.images.append(face)
                    self.labels.append(label)
                    count += 1
            else:
                print(f'Face not found in {image_adr}')
                continue
    
    def make_dataframe(self):
        self.append_dataframe(self.neg_faces, 0, augment=False)
        self.append_dataframe(self.deva_faces, 1, augment=True)
        self.append_dataframe(self.swar_faces, 2, augment=True)
        self.images = np.array(self.images)
        self.labels = np.array(self.labels)
        print('Images shape: ', self.images.shape)
        print('Labels shape: ', self.labels.shape)
        return self.images/255.0, self.labels, self.labels_list

x = dataframe()
images, labels, labels_list = x.make_dataframe()

Total images:  175
Count:  1  | Image address:  ./data/negative-faces/n000001/0001_01.jpg  | Label:  Unknown  | Image shape:  (227, 199, 3)
Count:  2  | Image address:  ./data/negative-faces/n000001/0002_01.jpg  | Label:  Unknown  | Image shape:  (511, 477, 3)
Count:  3  | Image address:  ./data/negative-faces/n000001/0003_01.jpg  | Label:  Unknown  | Image shape:  (190, 191, 3)
Count:  4  | Image address:  ./data/negative-faces/n000001/0004_01.jpg  | Label:  Unknown  | Image shape:  (600, 674, 3)
Count:  5  | Image address:  ./data/negative-faces/n000001/0005_01.jpg  | Label:  Unknown  | Image shape:  (199, 174, 3)
Count:  6  | Image address:  ./data/negative-faces/n000001/0006_01.jpg  | Label:  Unknown  | Image shape:  (878, 878, 3)
Count:  7  | Image address:  ./data/negative-faces/n000001/0007_01.jpg  | Label:  Unknown  | Image shape:  (532, 549, 3)
Count:  8  | Image address:  ./data/negative-faces/n000001/0008_01.jpg  | Label:  Unknown  | Image shape:  (295, 310, 3)
Count:  9  | 

In [ ]:
# for x in range(len(images)):
#     try:
#         plt.imshow(images[x])
#         plt.title(labels_list[int(labels[x])])
#         plt.show()
#     except:
#         print(f'Error in {x}')

## Splitting the data into train and test

In [9]:
# split the data into train and test
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2)

In [7]:
images.shape

(1545, 300, 300, 3)

In [8]:
train_images.shape

(1236, 300, 300, 3)

In [9]:
test_images.shape

(309, 300, 300, 3)

## Creating The CNN Model

In [6]:
model = models.Sequential()

model.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(300,300,3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Flatten())

model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

model.add(layers.Dense(512, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

model.add(layers.Dense(len(labels_list), activation='softmax'))


2023-02-22 22:16:47.120741: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-22 22:16:48.406687: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3358 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 128)     3584      
                                                                 
 batch_normalization (BatchN  (None, 298, 298, 128)    512       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 128)    0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 149, 149, 128)     0         
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 256)     295168    
                                                                 
 batch_normalization_1 (Batc  (None, 147, 147, 256)    1

In [12]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, 
                    epochs=10, 
                    validation_data=(test_images, test_labels))

2023-02-22 22:20:11.781334: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1334880000 exceeds 10% of free system memory.
2023-02-22 22:20:22.578753: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.24GiB (rounded to 1334880000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-02-22 22:20:22.578790: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2023-02-22 22:20:22.578802: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 43, Chunks in use: 43. 10.8KiB allocated for chunks. 10.8KiB in use in bin. 220B client-requested in use in bin.
2023-02-22 22:20:22.578811: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (512): 	Total Chunks: 11, Chunks in use: 11. 5.5K

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [14]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(test_acc)


2023-02-21 15:00:58.513725: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 333720000 exceeds 10% of free system memory.


10/10 - 3s - loss: 1.0077 - accuracy: 0.6472 - 3s/epoch - 256ms/step
0.647249162197113


In [ ]:
model.save('face_recognition_model_ep2.h5')

In [ ]:
model.load_weights('face_recognition_model_ep2.h5')

In [ ]:
def predict(model, labels_list, grayscale=False):
    # initialize the video capture object

    # read the frame from the webcam
    # frame = cv2.imread('data\Devasheesh\Devasheesh_32.jpg')
    while True:
        cap = cv2.VideoCapture(0)
        ret, frame = cap.read()
        frame = cv2.resize(frame, (300, 300))
        # extract the face
        face, (x, y, x2, y2) = extract_frontal_face(
            frame, grayscale=grayscale, size=(300, 300))
        if face is not None:
            # predict the face
            pred = model.predict(face.reshape(1, 300, 300, 3))
            # get the label
            label = labels_list[np.argmax(pred)]
            # draw the rectangle and put the text
            cv2.rectangle(frame, (x, y), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # show the frame
        cv2.imshow('Face Recognition', frame)

# call the predict function
predict(model, labels_list, grayscale=False)

# destroy all windows
cv2.destroyAllWindows()


In [1]:
# Test the model - Self independent test

# imports
import cv2
from tensorflow import keras
import numpy as np
# load the model
model = keras.models.load_model("face_recognition_model_ep2.h5")
# using DNN
modelFile = "dnn/res10_300x300_ssd_iter_140000.caffemodel"
configFile = "dnn/deploy.prototxt.txt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)


def extract_frontal_face(image_ndarray, size, grayscale=False):
    # resize the image to 300x300 for the DNN model
    h, w = 300, 300
    image_ndarray = cv2.resize(image_ndarray, (h, w))
    # Convert into blob
    blob = cv2.dnn.blobFromImage(
        image_ndarray, 1.0, (h, w), (104.0, 177.0, 123.0))
    # Convert into grayscale
    if grayscale:
        image_ndarray = cv2.cvtColor(image_ndarray, cv2.COLOR_BGR2GRAY)
    # Detect faces
    net.setInput(blob)
    detections = net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            # Get the coordinates of the bounding box
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x2, y2) = box.astype("int")

            # Extract the face ROI (region of interest) from the image
            face = image_ndarray[y:y2, x:x2]
            return cv2.resize(face, size), (x, y, x2, y2)
        else:
            return None, None


def predict_video(model, labels_list):
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        print(frame.shape)
        frame = cv2.resize(frame, (300, 300))
        try:
            # extract the face
            face, (x, y, x2, y2) = extract_frontal_face(
                frame, size=(300, 300))
            if face is not None:
                # predict the face
                pred = model.predict(face.reshape(1, 300, 300, 3))
                # get the label
                label = labels_list[np.argmax(pred)]
                # draw the rectangle and put the text
                cv2.rectangle(frame, (x, y), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x, y-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            # show the frame
            cv2.imshow('Face Recognition', frame)
            cv2.waitKey(1)
        except Exception as e:
            print(e)
            pass
       

# call the predict function
predict_video(model, labels_list=['Unknown', 'Devasheesh', 'Swarnim'])


2023-02-21 00:50:42.616118: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-21 00:50:44.789621: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/swarnim/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-02-21 00:50:44.789855: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/swarnim/.local/lib/python3.10/site-packages/cv2/../../lib

(480, 640, 3)


2023-02-21 00:50:52.103079: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8700
2023-02-21 00:50:52.871908: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1/1 [==============================] - 2s 2s/step
(480, 640, 3)
1/1 [==============================] - 0s 34ms/step
(480, 640, 3)
1/1 [==============================] - 0s 31ms/step
(480, 640, 3)
1/1 [==============================] - 0s 46ms/step
(480, 640, 3)
1/1 [==============================] - 0s 30ms/step
(480, 640, 3)
1/1 [==============================] - 0s 30ms/step
(480, 640, 3)
1/1 [==============================] - 0s 29ms/step
(480, 640, 3)
1/1 [==============================] - 0s 28ms/step
(480, 640, 3)
1/1 [==============================] - 0s 26ms/step
(480, 640, 3)
1/1 [==============================] - 0s 24ms/step
(480, 640, 3)
1/1 [==============================] - 0s 28ms/step
(480, 640, 3)
1/1 [==============================] - 0s 28ms/step
(480, 640, 3)
1/1 [==============================] - 0s 31ms/step
(480, 640, 3)
1/1 [==============================] - 0s 26ms/step
(480, 640, 3)
1/1 [==============================] - 0s 26ms/step
(480, 640, 3)
1/1 [=======

KeyboardInterrupt: 